In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()
(m_train, n_train) = env.get_training_data()
# Any results you write to the current directory are saved as output.

['marketdata_sample.csv', 'news_sample.csv']
Loading the data... This could take a minute.
Done!


In [2]:
print(m_train)

                             time   ...    universe
0       2007-02-01 22:00:00+00:00   ...         1.0
1       2007-02-01 22:00:00+00:00   ...         0.0
2       2007-02-01 22:00:00+00:00   ...         1.0
3       2007-02-01 22:00:00+00:00   ...         1.0
4       2007-02-01 22:00:00+00:00   ...         1.0
5       2007-02-01 22:00:00+00:00   ...         1.0
6       2007-02-01 22:00:00+00:00   ...         0.0
7       2007-02-01 22:00:00+00:00   ...         0.0
8       2007-02-01 22:00:00+00:00   ...         1.0
9       2007-02-01 22:00:00+00:00   ...         1.0
10      2007-02-01 22:00:00+00:00   ...         0.0
11      2007-02-01 22:00:00+00:00   ...         1.0
12      2007-02-01 22:00:00+00:00   ...         1.0
13      2007-02-01 22:00:00+00:00   ...         1.0
14      2007-02-01 22:00:00+00:00   ...         1.0
15      2007-02-01 22:00:00+00:00   ...         0.0
16      2007-02-01 22:00:00+00:00   ...         0.0
17      2007-02-01 22:00:00+00:00   ...         0.0
18      2007

In [3]:
m_train.time = m_train.time.dt.date
n_train.time = n_train.time.dt.hour
n_train.sourceTimestamp = n_train.sourceTimestamp.dt.hour
n_train.firstCreated = n_train.firstCreated.dt.date

In [4]:
n_train['assetCodesLen'] = n_train['assetCodes'].map(lambda x: len(eval(x)))
n_train['assetCodes'] = n_train['assetCodes'].map(lambda x: list(eval(x))[0])

In [5]:
grouping_cols = ['firstCreated', 'assetCodes']

In [6]:
n_train = n_train.groupby(grouping_cols, as_index=False).median()

In [ ]:
print(n_train)

In [7]:
m_train = pd.merge(m_train, n_train, how='left', left_on= ['time', 'assetCode'], right_on= ['firstCreated', 'assetCodes'])

In [ ]:
m_train

In [8]:
target = m_train.returnsOpenNextMktres10 >= 0

In [9]:
target = target.values

In [10]:
op = m_train.returnsOpenNextMktres10.values
op

array([ 0.03467204,  0.02780328,  0.02443291, ...,  0.01170343,
        0.08336656, -0.0162204 ])

In [11]:
reduced_cols = [c for c in m_train if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'time_x', 'universe','sourceTimestamp']]

In [ ]:
reduced_cols

In [12]:
final_dataset = m_train[reduced_cols].values

In [ ]:
final_dataset

In [13]:
from sklearn import *
X_train, X_test, target_train, target_test, op_train, op_test = model_selection.train_test_split(final_dataset, target, op, test_size=0.3, random_state=53)


In [14]:
from xgboost import XGBClassifier

In [18]:
xgb = XGBClassifier(n_jobs=2,n_estimators=100,max_depth=8,eta=0.3)

In [19]:
xgb.fit(X_train, target_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.3, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=2, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(xgb.predict(X_test),target_test)